# Notebook 2: Information Retrieval & Ranking

## Learning Objectives
By the end of this notebook, you will be able to:
- Define and compute TF, DF, and IDF from a small corpus
- Build TF–IDF vectors and rank documents using cosine similarity
- Explain and implement BM25 ranking, and compare it to TF–IDF
- Run hands-on experiments to see how tokenization and parameters affect ranking



## Outline
1. Setup and toy corpus
2. Tokenization and vocabulary
3. TF, DF, IDF
4. TF–IDF vectors and cosine similarity
5. BM25 ranking
6. TF–IDF vs BM25: comparisons
7. Hands-on exercises


In [ ]:
import math 
import numpy as np
import pandas as pd 

docs = [
    {"id": "D1", "text": "iphone 14 pro smartphone apple camera"},
    {"id": "D2", "text": "samsung galaxy smartphone android camera"},
    {"id": "D3", "text": "nike running shoes comfort"},
    {"id": "D4", "text": "adidas running shoes boost"},
    {"id": "D5", "text": "macbook pro laptop apple"}, ]

def tokenizer(text):
    return [t for t in str(text).lower().repalce('-', ' ').split() if t]




In [10]:
import math
import numpy as np
import pandas as pd

# Tiny corpus: product-like docs
docs = [
    {"id": "D1", "text": "iphone 14 pro smartphone apple camera"},
    {"id": "D2", "text": "samsung galaxy smartphone android camera"},
    {"id": "D3", "text": "nike running shoes comfort"},
    {"id": "D4", "text": "adidas running shoes boost"},
    {"id": "D5", "text": "macbook pro laptop apple"},
]

def tokenize(text: str):
    return [t for t in str(text).lower().replace('-', ' ').split() if t]

corpus_tokens = [tokenize(d["text"]) for d in docs]
corpus_df = pd.DataFrame({"id": [d["id"] for d in docs], "tokens": corpus_tokens})
corpus_df


,id,tokens
0,D1,"[iphone, 14, pro, smartphone, apple, camera]"
1,D2,"[samsung, galaxy, smartphone, android, camera]"
2,D3,"[nike, running, shoes, comfort]"
3,D4,"[adidas, running, shoes, boost]"
4,D5,"[macbook, pro, laptop, apple]"


In [ ]:
# Vocabulary and frequencies
from collections import Counter, defaultdict

vocab = sorted({t for tokens in corpus_tokens for t in tokens})
N = len(corpus_tokens)

doc_term_counts = [Counter(tokens) for tokens in corpus_tokens]
df_counts = Counter(t for tokens in corpus_tokens for t in set(tokens))

print("Vocab:", vocab)
print("Docs:", N)
print("DF counts:", dict(df_counts))


## TF, DF, IDF (formulas)

- Term Frequency (TF): count in a document (or normalized variant)
- Document Frequency (DF): number of documents containing the term
- Inverse Document Frequency (IDF):

$$\mathrm{IDF}(t) = \log\frac{N}{\mathrm{DF}(t)}$$

(We use natural log here; base choice only scales values.)


In [ ]:
# Compute IDF and TF-IDF vectors
idf = {t: math.log(N / df_counts[t]) for t in vocab}

# Build dense TF-IDF vectors in vocab order
def tfidf_vector(term_counts: Counter):
    return np.array([term_counts.get(t, 0) * idf[t] for t in vocab], dtype=float)

doc_vectors = [tfidf_vector(tc) for tc in doc_term_counts]

# Cosine similarity
def cosine(a: np.ndarray, b: np.ndarray):
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

# Build a query vector and rank
def rank_tfidf(query: str, top_k: int = 5):
    q_tokens = tokenize(query)
    q_counts = Counter(q_tokens)
    q_vec = tfidf_vector(q_counts)
    scores = []
    for doc, vec in zip(docs, doc_vectors):
        scores.append((doc["id"], cosine(q_vec, vec)))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

print("TF-IDF ranking for 'running shoes':", rank_tfidf("running shoes"))


## BM25 (formulas and intuition)

- Document length matters: longer docs naturally have more term hits; BM25 normalizes by length.
- Core scoring for a term t:

$$\mathrm{BM25}(t) = \mathrm{IDF}(t) \cdot \frac{tf \cdot (k_1 + 1)}{tf + k_1\cdot(1 - b + b\cdot\frac{len}{avglen})}$$

Where:
- tf: term frequency in this document
- len: document length (token count)
- avglen: average doc length in the corpus
- k1: term saturation (typ. 1.2-2.0)
- b: length normalization (typ. 0.6-0.9)


In [ ]:
# Implement BM25 ranking
avg_len = float(np.mean([len(toks) for toks in corpus_tokens]))
len_by_doc = {d["id"]: len(toks) for d, toks in zip(docs, corpus_tokens)}

# IDF variant used in BM25 (Okapi): add 0.5 smoothing to avoid negative/infinite values
bm25_idf = {}
for t in vocab:
    df = df_counts[t]
    bm25_idf[t] = math.log((N - df + 0.5) / (df + 0.5) + 1)


def bm25_score(query: str, k1: float = 1.5, b: float = 0.75):
    q_tokens = tokenize(query)
    q_counts = Counter(q_tokens)
    scores = {d["id"]: 0.0 for d in docs}
    for d, term_counts in zip(docs, doc_term_counts):
        Ld = len_by_doc[d["id"]]
        for t, qtf in q_counts.items():
            tf = term_counts.get(t, 0)
            if tf == 0:
                continue
            denom = tf + k1 * (1 - b + b * (Ld / avg_len))
            s = bm25_idf[t] * ((tf * (k1 + 1)) / denom)
            scores[d["id"]] += s
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked

print("BM25 ranking for 'running shoes':", bm25_score("running shoes")[:5])


In [ ]:
# Compare TF-IDF vs BM25 on a few queries
queries = ["iphone camera", "running shoes", "apple laptop", "android smartphone"]

for q in queries:
    tfidf_top = rank_tfidf(q)
    bm25_top = bm25_score(q)
    print(f"\nQuery: {q}")
    print("TF-IDF:", tfidf_top[:3])
    print("BM25  :", bm25_top[:3])


## Learning-to-Rank: Pointwise vs Pairwise vs Listwise (interview-ready)

- Pointwise: predict an absolute relevance score per (query, doc). Loss: regression or classification.
- Pairwise: learn preferences between doc pairs for the same query (d+ should rank above d−). Loss: pairwise logistic (RankNet), hinge.
- Listwise: optimize a list-level objective (e.g., soft-NDCG, ListNet/ListMLE, LambdaRank/LambdaMART).

Why pairwise?
- Directly models relative preferences; robust when absolute labels are noisy.
- Aligns with click-derived training data where we know clicked > non-clicked.
- Often simpler than listwise and captures ordering better than pointwise.



In [ ]:
# Pairwise demo: learn to prefer clicked docs using simple features
from itertools import combinations
from collections import defaultdict

# Create tiny synthetic click labels (for interview-style demo)
# Format: clicks[query][doc_id] = 1 if clicked else 0
clicks = defaultdict(dict)
clicks['running shoes']['D3'] = 1  # nike
clicks['running shoes']['D4'] = 1  # adidas
clicks['running shoes']['D1'] = 0
clicks['running shoes']['D2'] = 0
clicks['running shoes']['D5'] = 0

# Feature function: combine BM25 and TF-IDF cosine as two features

def features(query: str, doc_id: str):
    # TF-IDF cosine
    tfidf_score = dict(rank_tfidf(query)).get(doc_id, 0.0)
    # BM25 score
    bm25_score_map = dict(bm25_score(query))
    bm25_val = bm25_score_map.get(doc_id, 0.0)
    return np.array([tfidf_score, bm25_val], dtype=float)

# Build pairwise training data (d+ vs d- for same query)
X = []
y = []
for q, labels in clicks.items():
    pos = [d for d, lbl in labels.items() if lbl == 1]
    neg = [d for d, lbl in labels.items() if lbl == 0]
    for d_pos in pos:
        for d_neg in neg:
            # Positive direction: d_pos should outrank d_neg -> label 1
            X.append(features(q, d_pos) - features(q, d_neg))
            y.append(1)
            # Negative direction: d_neg should not outrank d_pos -> label 0
            X.append(features(q, d_neg) - features(q, d_pos))
            y.append(0)

X = np.vstack(X) if X else np.zeros((0, 2))
y = np.array(y) if y else np.zeros((0,), dtype=int)

# Pairwise logistic regression (RankNet-style on feature differences)
from sklearn.linear_model import LogisticRegression
clf = None
if len(np.unique(y)) >= 2:
    clf = LogisticRegression(solver='lbfgs')
    clf.fit(X, y)
    print("Pairwise weights (tfidf, bm25):", clf.coef_[0])
else:
    print("[Note] Not enough class variety for pairwise training; falling back to BM25-only ranking.")

# Inference: score docs with learned weights and rank

def pairwise_rank(query: str):
    # Fallback to BM25 if model not trained
    if clf is None:
        return bm25_score(query)
    scores = []
    w = clf.coef_[0]
    for d in docs:
        f = features(query, d['id'])
        scores.append((d['id'], float(np.dot(w, f))))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

print("\nPairwise model ranking for 'running shoes':", pairwise_rank('running shoes')[:5])


In [ ]:
# Build synthetic pointwise CTR dataset and train LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

pointwise_clicks = {
    'running shoes': {'D3': 1, 'D4': 1, 'D1': 0, 'D2': 0, 'D5': 0},
    'iphone camera': {'D1': 1, 'D2': 0, 'D3': 0, 'D4': 0, 'D5': 0},
    'apple laptop': {'D5': 1, 'D1': 1, 'D2': 0, 'D3': 0, 'D4': 0},
}

# Feature builder reused

def build_pointwise_dataset(clicks_dict):
    rows = []
    for q, label_map in clicks_dict.items():
        tfidf_map = dict(rank_tfidf(q))
        bm25_map = dict(bm25_score(q))
        for doc in docs:
            d = doc['id']
            if d not in label_map:
                continue
            x = {
                'query': q,
                'doc': d,
                'tfidf': tfidf_map.get(d, 0.0),
                'bm25': bm25_map.get(d, 0.0),
                'len': len_by_doc[d] / avg_len,
                'y': label_map[d]
            }
            rows.append(x)
    return pd.DataFrame(rows)

ctr_df = build_pointwise_dataset(pointwise_clicks)
X = ctr_df[['tfidf','bm25','len']].values
y = np.asarray(ctr_df['y'].values, dtype=int)

# Ensure both classes exist
if len(np.unique(y)) < 2:
    raise ValueError("Pointwise labels must contain both 0 and 1 for AUC/LogLoss.")

ctr_clf = LogisticRegression(solver='lbfgs', max_iter=1000)
ctr_clf.fit(X, y)
probs = ctr_clf.predict_proba(X)[:,1]

print("AUC:", f"{roc_auc_score(y, probs):.3f}")
print("LogLoss:", f"{log_loss(y, probs):.3f}")

# Per-query NDCG@3 using predicted CTR as ranking score

def dcg_at_k_labels(labels, k):
    dcg = 0.0
    for i, rel in enumerate(labels[:k]):
        dcg += (2**rel - 1) / math.log2(i + 2)
    return dcg

def ndcg_at_k_from_pred(df, k=3):
    ndcgs = []
    for q, group in df.groupby('query'):
        g = group.copy()
        g = g.assign(pred=ctr_clf.predict_proba(g[['tfidf','bm25','len']].values)[:,1])
        g_sorted = g.sort_values('pred', ascending=False)
        rel_sorted = g_sorted['y'].tolist()
        dcg = dcg_at_k_labels(rel_sorted, k)
        ideal = sorted(g['y'].tolist(), reverse=True)
        idcg = dcg_at_k_labels(ideal, k)
        ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcgs.append((q, ndcg))
    return ndcgs

nd = ndcg_at_k_from_pred(ctr_df, k=3)
print("Per-query NDCG@3:")
for q, v in nd:
    print(f"  {q}: {v:.3f}")
print("Avg NDCG@3:", f"{np.mean([v for _, v in nd]):.3f}")


### Exercises (Pairwise)
- Replace synthetic clicks with your own tiny labels; retrain and observe weight changes.
- Add a new feature (e.g., title exact-match flag) into `features()`; measure improvement.
- Stress test: make one doc much longer; compare TF-IDF vs BM25 vs pairwise ranking.
- Interview prompt: explain why pairwise training can learn to combine TF-IDF and BM25 better than either alone.


## Interview cheat sheet (IR & Ranking)

- BM25 intuition: term saturation (k1) prevents overcounting frequent terms; length norm (b) reduces bias for long docs; IDF down-weights common terms.
- TF‑IDF vs BM25: TF‑IDF is linear in TF and ignores document length; BM25 handles saturation and length, usually better for search.
- Learning‑to‑rank:
  - Pointwise: simple, but fits absolute labels; can misalign with ordering.
  - Pairwise: optimizes relative preferences; robust with click data.
  - Listwise: best alignment with ranking metrics (NDCG), more complex to train.
- Feature ideas: exact/phrase match, field boosts (title > description), popularity/freshness, price bands, brand match, clickthrough priors.
- Metric selection: use NDCG@K for overall ranking quality; MRR/Precision@K for navigational; Recall@K for discovery/coverage; always pair with online guardrails (CTR, Conversion, RPS, latency, zero‑results).
